In [ ]:
## Wonderful!!!
#!pip install jupyter-dash

In [1]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import dcc,html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [16]:
# Read the airline data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
#df.info()
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [21]:
site_pie_df=df.groupby(['Launch Site'])['class'].mean().reset_index()
site_pie_df

,Launch Site,class
0,CCAFS LC-40,0.269231
1,CCAFS SLC-40,0.428571
2,KSC LC-39A,0.769231
3,VAFB SLC-4E,0.400000


In [45]:
df['class'].value_counts(normalize=True).reset_index()


,index,class
0,0,0.571429
1,1,0.428571


In [32]:
scatter_payload_df=df.groupby(['Booster Version Category'])['class'].mean().reset_index()
#scatter_payload_df['class'].mean() # 0.45575757575757575
#df['class'].mean()               # 0.42857142857142855 --right
scatter_payload_df

,Booster Version Category,class
0,B4,0.545455
1,B5,1.000000
2,FT,0.666667
3,v1.0,0.000000
4,v1.1,0.066667


In [27]:
#df['Payload Mass (kg)'].unique() #48
df['Payload Mass (kg)'].max() #9600
df['Payload Mass (kg)'].min() # 0

0.0

In [56]:
site_list=df['Launch Site'].unique().tolist()
site_list

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import Input, Output, html, dcc
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 

                                            options=[
                                                {'label':'allSite' , 'value':'ALL' } ,
                                                {'label':'CCAFS LC-40' , 'value':'CCAFS LC-40' } ,
                                                {'label':'VAFB SLC-4E' , 'value':'VAFB SLC-4E' } ,
                                                {'label':'KSC LC-39A' , 'value':'KSC LC-39A' } ,
                                                {'label':'CCAFS SLC-40' , 'value':'CCAFS SLC-40' } ,
                                                    ],
                                                    value='ALL',
                                            placeholder="allSite",
                                            searchable=True,
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 
                                            'text-align-last' : 'center'},),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0', 100: '100'},
                                        value=[min_payload, max_payload]),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),  # output id respect the outlayer id.
    Input(component_id='site-dropdown', component_property='value'),
    )
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names='Launch Site', 
        title='title')
        return fig
    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]['class'].value_counts().reset_index(),
        fig = px.pie(df, values='class',
        names='class', 
        title='title')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider',component_property='value'),
)
def get_payload_scatter_chart(entered_site,slider_range):   # the paramter is combine the dropdown,and slider_range, how many input respect the paramter.
    filtered_df = spacex_df
    low, high = slider_range
    #mask = (df['petal_width'] > low) & (df['petal_width'] < high)
    #df = px.data.iris() # iris is a pandas DataFrame
    if entered_site == 'ALL':
        #pldf=spacex_df.loc[(spacex_df['Payload Mass (kg)']<=high) & (spacex_df['Payload Mass (kg)']>=low)]
        df = px.data.pldf()
        fig= px.scatter(spacex_df,y='class',x='Payload Mass (kg)',color='Booster Version Category')
        return fig

    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]
        pldf=df.loc[(df['Payload Mass (kg)']<=high) & (spacex_df['Payload Mass (kg)']>=low)]
        df = px.data.pldf()
        fig = px.catterie(df, y='class',x='Payload Mass (kg)', color='Booster Version Category',
        names='f{entered_site} Success Launch',
        title='title')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()